In [530]:
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [531]:
engine = create_engine("sqlite:///data/mta_turnstile.db")

In [532]:
all_tables = engine.table_names()
#all_tables

<ipython-input-532-454fd37e24c5>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  all_tables = engine.table_names()


In [533]:
mtadf = pd.read_sql('SELECT * FROM turnstile_2019_0105;',engine)

In [534]:
print('DTYPES:\n',mtadf.dtypes,'\n')
print(mtadf.columns)
print(mtadf.shape)

DTYPES:
 C/A                                                                     object
UNIT                                                                    object
SCP                                                                     object
STATION                                                                 object
LINENAME                                                                object
DIVISION                                                                object
DATE                                                                    object
TIME                                                                    object
DESC                                                                    object
ENTRIES                                                                  int64
EXITS                                                                    int64
dtype: object 

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',


In [535]:
# fix the insane amount of space after 'EXITS'
mtadf.rename(str.strip, axis='columns',inplace=True)
mtadf.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [536]:
# change DATE and TIME to datetime objects
mtadf['DATE']=pd.to_datetime(mtadf['DATE'])
mtadf['TIME']=pd.to_datetime(mtadf['TIME'],format='%H:%M:%S')

In [537]:
# create column for HOUR and MINUTE
mtadf['HOUR']=mtadf['TIME'].dt.hour
mtadf['MIN']=mtadf['TIME'].dt.minute

# drop the TIME column
mtadf.drop(columns='TIME',axis=1,inplace=True)

In [538]:
# create unique identifyer for each turnstile then drop original columns
mtadf['TURNSTILE'] = mtadf['C/A'] + '-' + mtadf['UNIT'] + '-' + mtadf['SCP']
mtadf.drop(['C/A','UNIT','SCP'],axis=1,inplace=True)

# rearrange order of columns
col_names = ['STATION','TURNSTILE','DATE','HOUR','MIN','ENTRIES','EXITS','LINENAME','DIVISION','DESC']
mtadf = mtadf.reindex(columns=col_names)

In [539]:
# check for null values in dataset -- no null values
mtadf.isna().sum()

STATION      0
TURNSTILE    0
DATE         0
HOUR         0
MIN          0
ENTRIES      0
EXITS        0
LINENAME     0
DIVISION     0
DESC         0
dtype: int64

In [540]:
# sort the database by station, then turnstile, then date, then time
mtadfs = mtadf.sort_values(['STATION','TURNSTILE','DATE','HOUR','MIN'])

# which 10 stations have the least amount of turnstiles?
mtadfs.groupby(['STATION'])['TURNSTILE'].nunique().nsmallest(10)

# create a database with 5 stations
mtadfs_3 = mtadfs[mtadfs['STATION'].isin(['138/GRAND CONC','182-183 STS','190 ST'])]

In [541]:
# snippet from above (when isolating 1 day)
                       # &mtadfs['DATE'].isin(['2019-01-04'])]
    
# check max amount of entries per station
# CLEVELAND ST and SUTTER AV have significantly less entries
# remove them from above .isin() statement
mtadfs_3.groupby(['STATION'])['ENTRIES'].max()

# 3 turnstiles per STATION
mtadfs_3.groupby(['STATION'])[['TURNSTILE']].describe()

# 6 entries per day per TURNSTILE
mtadfs_3['TURNSTILE'].sort_values().value_counts()

N215-R237-00-00-02     42
N215-R237-00-00-00     42
R259-R307-00-00-01     42
N215-R237-00-00-01     42
N006A-R280-00-00-00    42
N006A-R280-00-00-01    42
N006A-R280-00-00-02    42
R259-R307-00-00-00     42
R259-R307-00-00-02     42
Name: TURNSTILE, dtype: int64

In [542]:
# we now have a database with information 
# for 3 STATIONS (138/GRAND CONC, 182-183 STS, 190 ST)
mtadfs_3['DATE'].dt.day.value_counts()

# 9 unique TURNSTILES total
# 6 entries per day per TURNSTILE
# 7 days
# 42 total entries per TURNSTILE

1     54
2     54
3     54
4     54
29    54
30    54
31    54
Name: DATE, dtype: int64

In [543]:
# create a database without the unnecessary columns and reset index
mtadfs_3d=mtadfs_3[['STATION',
                                'TURNSTILE',
                                'LINENAME',
                                'DATE',
                                'HOUR',
                                'MIN',
                                'ENTRIES',
                                'EXITS']].reset_index()

# drop the old indexes from before the sort
mtadfs_3d.drop('index',axis=1,inplace=True)

In [544]:
# calculate the sum of the entries and exits per time period
# as long as it is in order by time and day, cumulative entries should always go up
mtadfs_3d['SUM_ENTRIES'] = mtadfs_3d.groupby(['TURNSTILE'])['ENTRIES'].diff()
mtadfs_3d['SUM_EXITS'] = mtadfs_3d.groupby(['TURNSTILE'])['EXITS'].diff()

# shift the column data up by 1 row (periods=-1)
mtadfs_3d['SUM_ENTRIES'] = mtadfs_3d['SUM_ENTRIES'].shift(periods=-1, axis=0)
mtadfs_3d['SUM_EXITS'] = mtadfs_3d['SUM_EXITS'].shift(periods=-1, axis=0)

# shape = 378 rows
mtadfs_3d.shape

(378, 10)

In [545]:
# double check for negative values in the SUM_ENTRIES (if it is cumulative all values should be positive)
mtadfs_3d_nonegs = mtadfs_3d[mtadfs_3d['SUM_ENTRIES'] > 0]

# shape check = 287 rows
mtadfs_3d_nonegs.shape

# double check for negative values in the SUM_EXITS (if it is cumulative all values should be positive)
mtadfs_3d_nonegs = mtadfs_3d_nonegs[mtadfs_3d_nonegs['SUM_EXITS'] > 0]

#shape check = 286 rows
mtadfs_3d_nonegs.shape

# alternate syntax
# nonegs = (mtadfs_3d['SUM_ENTRIES'] > 0) | (mtadfs_3d['SUM_EXITS'] > 0)
# mtadfs_3d_nonegs = mtadfs_3d[nonegs]

(286, 10)

In [546]:
# create a column to indicate how 'busy a station is during that interval'
mtadfs_3d_nonegs['BUSY'] = mtadfs_3d_nonegs['SUM_ENTRIES'] + mtadfs_3d_nonegs['SUM_EXITS']

In [547]:
mtadfs_3d_nonegs.head(20)

,STATION,TURNSTILE,LINENAME,DATE,HOUR,MIN,ENTRIES,EXITS,SUM_ENTRIES,SUM_EXITS,BUSY
0,138/GRAND CONC,R259-R307-00-00-00,45,2018-12-29,3,0,3593616,3343021,87.0,83.0,170.0
1,138/GRAND CONC,R259-R307-00-00-00,45,2018-12-29,7,0,3593703,3343104,166.0,141.0,307.0
2,138/GRAND CONC,R259-R307-00-00-00,45,2018-12-29,11,0,3593869,3343245,199.0,77.0,276.0
3,138/GRAND CONC,R259-R307-00-00-00,45,2018-12-29,15,0,3594068,3343322,215.0,147.0,362.0
4,138/GRAND CONC,R259-R307-00-00-00,45,2018-12-29,19,0,3594283,3343469,109.0,105.0,214.0
5,138/GRAND CONC,R259-R307-00-00-00,45,2018-12-29,23,0,3594392,3343574,51.0,77.0,128.0
6,138/GRAND CONC,R259-R307-00-00-00,45,2018-12-30,3,0,3594443,3343651,86.0,55.0,141.0
7,138/GRAND CONC,R259-R307-00-00-00,45,2018-12-30,7,0,3594529,3343706,95.0,65.0,160.0
8,138/GRAND CONC,R259-R307-00-00-00,45,2018-12-30,11,0,3594624,3343771,165.0,51.0,216.0
9,138/GRAND CONC,R259-R307-00-00-00,45,2018-12-30,15,0,3594789,3343822,124.0,112.0,236.0


In [548]:
# turn the lines into categorical data
# first split up the string to have one letter/number per column
splits = mtadfs_3d_nonegs['LINENAME'].str.split('',expand=True)

# look at spits
splits

# it didn't create a column per unique line

,0,1,2,3
0,,4,5,
1,,4,5,
2,,4,5,
3,,4,5,
4,,4,5,
...,...,...,...,...
372,,A,,None
373,,A,,None
374,,A,,None
375,,A,,None


In [549]:
# double check what has happened
print(splits.loc[:,0].unique())
print(splits.loc[:,1].unique())
print(splits.loc[:,2].unique())
print(splits.loc[:,3].unique())

# column location 0 and column location 3 don't contain anything

['']
['4' 'B' 'A']
['5' 'D' '']
['' None]


In [550]:
# drop column location 0 and column location 3
splits.drop(labels=[0,3], axis=1,inplace=True)
splits

# column names are integers, use .loc to indicate 0:3 is not the index but the column name
# get dummies
dummy=pd.get_dummies(splits.loc[:,1:2])

In [551]:
# we should have FIVE columns: A, B, D, 4, 5
# but we found an extra column called '2_'
dummy.drop(columns='2_',axis=1,inplace=True)

In [554]:
# lets rename these to make more sense
linemap = {'1_4':'LINE_4','1_A':'LINE_A','1_B':'LINE_B','2_5':'LINE_5','2_D':'LINE_D'}
dummy.rename(mapper=linemap,axis=1,inplace=True)

# add back into our dataframe
mtadfs_3d_nonegs_lines = pd.concat([mtadfs_3d_nonegs,dummy],axis=1)
mtadfs_3d_nonegs_lines.drop('LINENAME',axis=1,inplace=True)

In [556]:
mtadfs_3d_nonegs
mtadfs_3d_nonegs_lines

,STATION,TURNSTILE,DATE,HOUR,MIN,ENTRIES,EXITS,SUM_ENTRIES,SUM_EXITS,BUSY,LINE_4,LINE_A,LINE_B,LINE_5,LINE_D
0,138/GRAND CONC,R259-R307-00-00-00,2018-12-29,3,0,3593616,3343021,87.0,83.0,170.0,1,0,0,1,0
1,138/GRAND CONC,R259-R307-00-00-00,2018-12-29,7,0,3593703,3343104,166.0,141.0,307.0,1,0,0,1,0
2,138/GRAND CONC,R259-R307-00-00-00,2018-12-29,11,0,3593869,3343245,199.0,77.0,276.0,1,0,0,1,0
3,138/GRAND CONC,R259-R307-00-00-00,2018-12-29,15,0,3594068,3343322,215.0,147.0,362.0,1,0,0,1,0
4,138/GRAND CONC,R259-R307-00-00-00,2018-12-29,19,0,3594283,3343469,109.0,105.0,214.0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,190 ST,N006A-R280-00-00-02,2019-01-04,3,0,7150918,975954,100.0,4.0,104.0,0,1,0,0,0
373,190 ST,N006A-R280-00-00-02,2019-01-04,7,0,7151018,975958,663.0,26.0,689.0,0,1,0,0,0
374,190 ST,N006A-R280-00-00-02,2019-01-04,11,0,7151681,975984,187.0,24.0,211.0,0,1,0,0,0
375,190 ST,N006A-R280-00-00-02,2019-01-04,15,0,7151868,976008,252.0,57.0,309.0,0,1,0,0,0
